In [ ]:
from rlhfutils.eval_utils import oai_kwargs, load_alldfs, annotate_apfarm, apf_format, load_wgpt, filter_and_sort_df
import pandas as pd
from statistics import mean
import matplotlib.pyplot as plt
import re
from transformers import AutoTokenizer
from datasets import load_dataset
import openai
from rlhfutils.data import qaform
import random

In [ ]:
def stattest (perc, amnt):
    lis = [0]*int(amnt*(1-perc)) + [1]*int(amnt*perc)
    print(len(lis))
    dis = []
    for i in range(10000):
        tmp = random.choices(lis, k=len(lis))
        if mean(tmp)>0.5:
            dis.append(1)
        else:
            dis.append(0)
    return mean(dis)

In [ ]:
stattest(0.53, 1000)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#stackout = pd.read_json("../trl-general/generated_wgptppoorig.jsonl", orient='records', lines=True)
toker = AutoTokenizer.from_pretrained("../webgpt-llama/models/sft10k")

In [ ]:
s = """
Adults typically do not
roll off of the bed be-
cause they have devel-
oped the muscle mem-
ory to keep their bod-
ies from involuntarily
moving during sleep
and maintaining proper
posture
"""
len(toker(s).input_ids)

In [ ]:
tdset = load_dataset("openai/webgpt_comparisons", split="train")
def preproc_wgpt(example):
    ex = {}
    ex['question'] = example['question']['full_text']
    if example['score_0']>example['score_1']:
        ex['response_j'] = example['answer_0']
        ex['response_k'] = example['answer_1']
    else:
        ex['response_k'] = example['answer_0']
        ex['response_j'] = example['answer_1']
    return ex
tdset = tdset.map(preproc_wgpt)

In [ ]:
res = []
lqs = list(adfs['wgptppoorig'].question)
for t in tdset: 
    if t['question'] in lqs:
        res.append({
            'question':t['question'],
            'response':t['response_j']
        })

In [ ]:
adfs.keys()

In [ ]:
adfs['stackbalance'].rcnt.mean()

In [ ]:
toker.decode("Answer:")

In [ ]:
toker(stackout['response'][1]).input_ids.index(13291)

In [ ]:
adfs = load_alldfs("../trl-general/genouts/lastevalsstack//",500 )

In [ ]:
adfs['stackorigrerun']

In [ ]:
adfs['rlcddashort'].rcnt.mean()

In [ ]:
import re

def process_str(text):
    # Regular expression to match URLs
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    # Replace all URLs in the text with "URL"
    s =  url_pattern.sub("[URL]", text)
    escapes = ["\n", "\\", "#",  "&", "%"]
    for e in escapes:
        s = s.replace(e, "")
    s = s.replace("\n", "").replace("#", "\#").replace("&", "\&").replace("%", "\%")
    return s

In [ ]:
ind = 40
s1 = process_str(adfs['rlcdsftv3'].loc[ind]['question'])
s1 = s1+" &\n"
s1 = s1+process_str(adfs['rlcdsftv3'].loc[ind]['response'])
s1 = s1+" &\n"
s1 = s1+process_str(adfs['rlcdorigv3'].loc[ind]['response'])
print(s1)

In [ ]:
adfs['wgptrwscale']

In [ ]:
mean(adfs['stacklenonly'].rcnt)

In [ ]:
ind = 3
print(adfs['rlcdsft'].loc[ind]['question'])
print(adfs['rlcdsft'].loc[ind]['response'])

In [ ]:
refdf = load_wgpt(200,0)
refdf =  pd.DataFrame(refdf)
refdf['question'] = [s.strip() for s in refdf['question']]
refdf = filter_and_sort_df(adfs['wgptppoorig'], refdf)

In [ ]:
adfs.keys()

In [ ]:
mean(adfs['stacklenonly'].rcnt)

In [ ]:
tmpdf['response'].loc[25]

In [ ]:
adfs['webgptdappov2125']['response'][1]

In [ ]:
everything = pd.concat(list(adfs.values()))

In [ ]:
adfs['stackbalance']

In [ ]:
adfs.keys()

In [ ]:
prompts = ["You are a helpful assistant. Answer the following question. Question: "+s+"\n\nAssistant: " for s in adfs['wgptsft']['question']]

In [ ]:
davdf = pd.DataFrame({"question":adfs['wgptsft']['question'], 'response':completions})

In [ ]:
davdf.to_json("../trl-general/genouts/davinciwebgpt.jsonl", orient='records',  lines=True)

In [ ]:
davdf

In [ ]:
adfs['davinci']['rcnt'] = 

In [ ]:
# completions = []
# for prompt in prompts:
#     response = openai.Completion.create(
#       model="text-davinci-003",
#       prompt=prompt,
#       max_tokens=256,  # Adjust based on your requirements
#       top_p=0.9,
#       temperature=0.9
#     )
#     completions.append(response.choices[0].text.strip())
#     print("__")
#     print(completions[-1])

In [ ]:
adfs = load_alldfs("../trl-general/genouts/wgpt/")

In [ ]:
adfs.keys()

In [ ]:
adfs.keys()

In [ ]:
# compare all to ppo original
lenannot = annotate_apfarm(adfs, "wgptsft2", "wgptppoorig2", 100, len(adfs['wgptsft2']), oai_kwargs())

In [ ]:
tmp = pd.DataFrame(lenannot)

In [ ]:
tmp.preference.mean()

In [ ]:
ind = 8
print(adfs['apfgoodcut']['question'][ind])
print(adfs['apfgoodcut']['response'][ind])

In [ ]:
tmp = pd.read_json("../outputs/apeval/wgptsft2_wgptppoorig2_500.jsonl", orient='records', lines=True)

In [ ]:
alls = []
for i in range(100, 397, 100):
    alls.append(pd.read_json("../outputs/apeval/stacklenonlyppo3_stackorigrerun_"+str(i)+".jsonl", orient='records', lines=True))
asdfs = pd.concat(alls)

In [ ]:
len(asdfs[asdfs.preference==2]) + len(asdfs[asdfs.preference==1])

In [ ]:
(1-(asdfs.preference.mean()-1))

In [ ]:
tmpdf = pd.read_json("../outputs/apeval/stackorigrerun_stackbalance.jsonl", orient='records', lines=True)

In [ ]:
tmpdf.preference.mean()

In [ ]:
list(tmp.preference)

In [ ]:
adfs['davinci']

In [ ]:
everything = pd.concat(list(adfs.values()))
assert len(set(everything['question']))==len(everything)/len(adfs.keys())
assert len(set(everything['response']))==len(everything)
assert [len(set(a['question']))==len(a[''])]
assert
print(len(everything)/len(adfs.keys()))

In [ ]:
[len(a) for a in adfs.values()] 

In [ ]:
adfs['davinci']

In [ ]:
len(adfs['chosen'])